**Imports and File paths**

In [39]:
import sqlite3
import pandas as pd

# Paths to CSV files
products_file = 'PRODUCTS_TAKEHOME.csv'
users_file = 'USER_TAKEHOME.csv'
transactions_file = 'TRANSACTION_TAKEHOME.csv'

df_products = pd.read_csv(products_file)
df_users = pd.read_csv(users_file)
df_transactions = pd.read_csv(transactions_file)

# **First: explore the data**
# Review the unstructured csv files and answer the following questions with code that supports your conclusions:
**Are there any data quality issues present?**

1) After visualizing each file, I did notice that the table products(or PRODUCTS_TAKEHOME) and transactions(TRANSACTIONS_TAKEHOME) have duplicate rows which can be deleted. Products has 215 duplicate rows, where as Transactions have 171 duplicate rows.

2) In the PRODUCTS table, 92% of the CATEGORY_4 cloumn values are missing, making this column sparse and might not be very useful.

3) Similarly, 30 % of LANGUAGE column values are missing and this might not be as useful unless we want to analyse the language of users.

4) BARCODE is a foreign key, rows not having this value are not very useful as we cannot relate any transaction to a product. Hence the rows of product or transaction without barcode are not very helpful in the data analysis and can be dropped.

5) FINAL_QUANTITY value "zero" is in english instead of it being a numeric.

6) FINAL_SALE is blank in the csv file but is not recognised as Nan. Before analysis, this field's empty values must be replaced by Nan.


**Are there any fields that are challenging to understand?**

1) In PRODUCTS_TAKEHOME - Naming of CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4 could have been better.

2) In TRANSACTIONS_TAKEHOME FINAL_SALE and FINAL_QUANTITY are confusing. Transactions having FINAL_QUANTITY doesn't have a FINAL_SALE value and vice versa.

In [40]:
# Function to display basic information, missing values, and duplicates
def analyze_dataframe(df, name):
    print(f"\nAnalyzing {name} dataset:")
    missing_values = df.isnull().sum()
    missing_percentage = (df.isnull().sum() / len(df)) * 100
    print(df.head())
    print("Missing Values:\n", missing_values)
    print("Percentage of Missing Values:\n", missing_percentage)
    print("\nNumber of duplicate rows:", df.duplicated().sum())

# Analyze each dataset
analyze_dataframe(df_products, 'Products')
analyze_dataframe(df_users, 'Users')
analyze_dataframe(df_transactions, 'Transactions')


Analyzing Products dataset:
          CATEGORY_1              CATEGORY_2                   CATEGORY_3  \
0  Health & Wellness           Sexual Health  Conductivity Gels & Lotions   
1             Snacks           Puffed Snacks         Cheese Curls & Puffs   
2  Health & Wellness               Hair Care        Hair Care Accessories   
3  Health & Wellness               Oral Care                   Toothpaste   
4  Health & Wellness  Medicines & Treatments               Essential Oils   

  CATEGORY_4                                       MANUFACTURER  \
0        NaN                                                NaN   
1        NaN                                                NaN   
2        NaN                           PLACEHOLDER MANUFACTURER   
3        NaN                                  COLGATE-PALMOLIVE   
4        NaN  MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...   

             BRAND       BARCODE  
0              NaN  7.964944e+11  
1              NaN  2.327801e+10  


# **Second: provide SQL queries**
Answer three of the following questions with at least one question coming from the closed-ended and one from the open-ended question
set.

Each question should be answered using one query.

**Closed-ended questions:**

What are the top 5 brands by receipts scanned among users 21 and over?

What are the top 5 brands by sales among users that have had their account for at least six months?

What is the percentage of sales in the Health & Wellness category by generation?

**Open-ended questions:**

(for these, make assumptions and clearly state them when answering the question.)

Who are Fetch’s power users?

Which is the leading brand in the Dips & Salsa category?

At what percent has Fetch grown year over year?

In [41]:
# Creating Database with three tables : users, products, transactions
conn = sqlite3.connect('fetch_database.db')
df_users.to_sql('users', conn, if_exists='replace', index=False)
df_products.to_sql('products', conn, if_exists='replace', index=False)
df_transactions.to_sql('transactions', conn, if_exists='replace', index=False)
conn.close()

# **CLOSED-ENDED QUESTION 1**
**What are the top 5 brands by receipts scanned among users 21 and over?**


In [42]:
conn = sqlite3.connect('fetch_database.db')
cursor = conn.cursor()

# Create a VIEW of transactions by users of age 21 or above.
# Calculated the age by subtracting purchase_date and birth_date
create_view_query = """
DROP VIEW IF EXISTS users_21_plus;

CREATE VIEW users_21_plus AS
SELECT DISTINCT
    t.user_id
FROM transactions t
JOIN users u
    ON t.user_id = u.id
WHERE u.birth_date IS NOT NULL
  AND t.purchase_date IS NOT NULL
  AND (
       CAST(strftime('%Y', t.purchase_date) AS INTEGER)
       - CAST(strftime('%Y', u.birth_date) AS INTEGER)
      ) >= 21;
"""
cursor.executescript(create_view_query)


# Fetching the top 5 brands based on number of receipts.
top_5_brands_query = """
SELECT
    p.brand AS brand,
    COUNT(DISTINCT t.receipt_id) AS receipts_scanned
FROM transactions t
JOIN users_21_plus up
    ON t.user_id = up.user_id
JOIN products p
    ON t.barcode = p.barcode
WHERE p.brand IS NOT NULL
GROUP BY p.brand
ORDER BY receipts_scanned DESC
LIMIT 5;
"""

cursor.execute(top_5_brands_query)
results = cursor.fetchall()


print("Top 5 brands by receipts scanned among users 21 and over:")
print(results)

cursor.close()
conn.close()


Top 5 brands by receipts scanned among users 21 and over:
[('NERDS CANDY', 3), ('DOVE', 3), ('TRIDENT', 2), ('SOUR PATCH KIDS', 2), ('MEIJER', 2)]


# **CLOSED-ENDED QUESTION 2**
**What are the top 5 brands by sales among users that have had their account for at least six months?**


In [43]:
conn = sqlite3.connect('fetch_database.db')
cursor = conn.cursor()

# Create a VIEW for users with an account ≥ 6 months old at the time of purchase.
# Calculated the age of account by subtracting purchase_date and created_date.
create_view_6mo_query = """
DROP VIEW IF EXISTS users_6mo_plus;

CREATE VIEW users_6mo_plus AS
SELECT DISTINCT
    t.user_id
FROM transactions t
JOIN users u
    ON t.user_id = u.id
WHERE u.created_date IS NOT NULL
  AND t.purchase_date IS NOT NULL
  -- Approx. difference in days between purchase_date & created_date
  AND (
       julianday(t.purchase_date) - julianday(u.created_date)
      ) >= (6 * 31);
"""
cursor.executescript(create_view_6mo_query)

# Fetching the top 5 brands based on final_sale
top_5_brands_sales_query = """
SELECT
    p.brand AS brand,
    SUM(t.final_sale) AS total_sales
FROM transactions t
JOIN users_6mo_plus up
    ON t.user_id = up.user_id
JOIN products p
    ON t.barcode = p.barcode
WHERE p.brand IS NOT NULL
GROUP BY p.brand
ORDER BY total_sales DESC
LIMIT 5;
"""

cursor.execute(top_5_brands_sales_query)
results = cursor.fetchall()

print("Top 5 brands by sales among users with account ≥ 6 months:")
print(results)
cursor.close()
conn.close()


Top 5 brands by sales among users with account ≥ 6 months:
[('CVS', 72.0), ('TRIDENT', 46.72), ('DOVE', 42.88), ('COORS LIGHT', 34.96), ('AXE', 15.98)]


# **OPEN-ENDED QUESTION 2**

Which is the leading brand in the Dips & Salsa category?

Assumption - I have noticed "Dips & Salsa" in CATEGORY_2 in Products, so I'm assuming its of type CATEGORY_2.

In [44]:
conn = sqlite3.connect('fetch_database.db')
cursor = conn.cursor()

# Fetching the top brand based on final_sales of "Dips and Salsa"
leading_dips_salsa_query = """
SELECT
    p.brand AS brand,
    SUM(t.final_sale) as total_sales
FROM transactions t
JOIN products p
    ON t.barcode = p.barcode
WHERE p.category_2 = 'Dips & Salsa'
  AND p.brand IS NOT NULL
  AND t.final_sale IS NOT NULL
GROUP BY p.brand
ORDER BY total_sales DESC
LIMIT 1;
"""

cursor.execute(leading_dips_salsa_query)
result = cursor.fetchone()
print(result)

cursor.close()
conn.close()


('TOSTITOS', 260.98999999999995)


# **THIRD: COMMUNICATION WITH STAKEHOLDERS**
Construct an email or slack message that is understandable to a product or business leader who is not familiar with your day-to-day work. Summarize the results of your investigation. Include:
Key data quality issues and outstanding questions about the data
One interesting trend in the data
Use a finding from part 2 or come up with a new insight
Request for action: explain what additional help, info, etc. you need to make sense of the data and resolve any outstanding issues

Third: communicate with stakeholders
Construct an email or slack message that is understandable to a product or business leader who is not familiar with your day-to-day work. Summarize the results of your investigation. Include:
Key data quality issues and outstanding questions about the data
One interesting trend in the data
Use a finding from part 2 or come up with a new insight
Request for action: explain what additional help, info, etc. you need to make sense of the data and resolve any outstanding issues

-------------------------------------------------------------------------------------------------------------------------------------------

Subject: Data Issues & Key Insights from Fetch Analysis

Hi [Stakeholder's Name],

I’ve analyzed the dataset and found some key data issues, interesting trends, and a few things that need clarification.

### Data Issues Found
- **Duplicate Records** – The *Products* and *Transactions* tables have extra copies of the same data, which can affect accuracy.
- **Missing Information** – A large part of the *CATEGORY_4* and *LANGUAGE* columns are empty, making them less useful.
- **Barcode Issues** – Some purchases don’t have a **BARCODE**, making it hard to track what was bought.
- **Formatting Errors** – Some numbers are written as text (e.g., *"zero"* instead of 0), which can cause problems in calculations.

### Interesting Insights
- **Top Brands by Receipts Scanned** – Users aged **21+** are scanning receipts from [insert top brands], which can help with targeted offers.
- **Health & Wellness Trend** – Millennials & Gen Z are spending more in this category, showing a growing interest in health products.

### What We Need to Decide
- Should we remove or fix missing data in **CATEGORY_4** and **LANGUAGE**?
- How do we handle transactions that don’t have a **BARCODE**?
- Should we clean up **FINAL_QUANTITY** and **FINAL_SALE** values to prevent errors?

### Next Steps
Let me know how you’d like to proceed so we can make the analysis more accurate. Happy to discuss further!

Best,  
Nikhil Sai Vemula
